# CutLER - Entraînement sur Dataset Personnalisé

Ce notebook démontre comment utiliser CutLER pour entraîner un modèle de détection d'objets sur votre propre dataset.

## Fonctionnalités ajoutées :

1. **Génération automatique de datasets synthétiques** - Crée des images avec formes géométriques
2. **Support de datasets personnalisés** - Compatible avec le format COCO
3. **Scripts d'entraînement adaptés** - Configuration automatique pour votre dataset
4. **Outils de visualisation** - Comparaison ground truth vs prédictions
5. **Pipeline complet** - De la génération des données à l'évaluation

## Structure des fichiers créés :

- `tools/generate_custom_dataset.py` - Génère un dataset synthétique
- `tools/train_custom.py` - Entraîne le modèle sur votre dataset
- `tools/visualize_custom.py` - Visualise les résultats
- `tools/run_custom_pipeline.py` - Script principal
- `cutler/data/datasets/custom_datasets.py` - Support pour datasets personnalisés
- `cutler/model_zoo/configs/Custom-Dataset/` - Configuration pour datasets personnalisés

In [ ]:
import torch
import os
import sys
import subprocess
from pathlib import Path

# Ajouter le répertoire du projet au PYTHONPATH
project_root = Path.cwd()
sys.path.insert(0, str(project_root))

print(f"Répertoire du projet: {project_root}")
print(f"CUDA disponible: {torch.cuda.is_available()}")

# Vérifier que les scripts personnalisés existent
tools_dir = project_root / "tools"
custom_scripts = [
    "generate_custom_dataset.py",
    "train_custom.py",
    "visualize_custom.py",
    "run_custom_pipeline.py"
]

print("\nScripts personnalisés:")
for script in custom_scripts:
    script_path = tools_dir / script
    status = "✓" if script_path.exists() else "✗"
    print(f"{status} {script}")

In [ ]:
# Démonstration 1: Génération d'un dataset personnalisé

# Créer un répertoire de test
workspace_dir = project_root / "test_workspace"
workspace_dir.mkdir(exist_ok=True)

dataset_dir = workspace_dir / "custom_dataset"

print(f"Génération d'un dataset d'exemple dans: {dataset_dir}")

# Commande pour générer le dataset
cmd = [
    sys.executable,
    str(tools_dir / "generate_custom_dataset.py"),
    "--output-dir", str(dataset_dir),
    "--num-train", "20",  # Petit dataset pour test
    "--num-val", "5"
]

try:
    result = subprocess.run(cmd, capture_output=True, text=True, check=True)
    print("✓ Dataset généré avec succès!")
    print("\nStructure créée:")

    # Afficher la structure du dataset
    for root, dirs, files in os.walk(dataset_dir):
        level = root.replace(str(dataset_dir), '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files[:3]:  # Limiter l'affichage
            print(f"{subindent}{file}")
        if len(files) > 3:
            print(f"{subindent}... et {len(files)-3} autres fichiers")

except subprocess.CalledProcessError as e:
    print(f"✗ Erreur: {e}")
    print(f"Sortie: {e.stdout}")
    print(f"Erreur: {e.stderr}")

True

In [ ]:
# Démonstration 2: Visualisation du dataset généré

if dataset_dir.exists():
    print("Visualisation des annotations ground truth...")

    visualization_dir = workspace_dir / "visualizations"

    cmd = [
        sys.executable,
        str(tools_dir / "visualize_custom.py"),
        "--dataset-path", str(dataset_dir),
        "--output-dir", str(visualization_dir),
        "--mode", "gt",
        "--num-images", "3",
        "--split", "train"
    ]

    try:
        result = subprocess.run(
            cmd, capture_output=True, text=True, check=True)
        print("✓ Visualisations créées!")

        # Lister les fichiers de visualisation créés
        if visualization_dir.exists():
            viz_files = list(visualization_dir.glob("*.png"))
            print(f"\nFichiers de visualisation créés: {len(viz_files)}")
            for file in viz_files:
                print(f"  - {file.name}")

    except subprocess.CalledProcessError as e:
        print(f"✗ Erreur lors de la visualisation: {e}")
        print(f"Sortie: {e.stdout}")
        print(f"Erreur: {e.stderr}")

else:
    print("Dataset non trouvé. Exécutez d'abord la cellule de génération.")

SyntaxError: invalid syntax (3797704349.py, line 1)

In [ ]:
# Démonstration 3: Pipeline complet (génération + entraînement + visualisation)

print("Exécution du pipeline complet avec le script principal...")

# Créer un nouveau workspace pour le test complet
full_workspace = project_root / "full_test_workspace"

cmd = [
    sys.executable,
    str(tools_dir / "run_custom_pipeline.py"),
    "--workspace", str(full_workspace),
    "--num-train", "10",  # Très petit pour test rapide
    "--num-val", "3",
    "--num-classes", "3",
    "--skip-training"  # Ignorer l'entraînement pour ce test (long)
]

try:
    print("Commande exécutée:")
    print(" ".join(cmd))
    print("\nExécution en cours...")

    result = subprocess.run(cmd, capture_output=True, text=True, check=True)
    print("✓ Pipeline exécuté avec succès!")
    print("\nSortie:")
    print(result.stdout)

    # Afficher la structure créée
    if full_workspace.exists():
        print(f"\nStructure créée dans {full_workspace}:")
        for item in full_workspace.iterdir():
            if item.is_dir():
                print(f"  📁 {item.name}/")
                for subitem in item.iterdir():
                    if subitem.is_dir():
                        print(f"    📁 {subitem.name}/")
                    else:
                        print(f"    📄 {subitem.name}")
            else:
                print(f"  📄 {item.name}")

except subprocess.CalledProcessError as e:
    print(f"✗ Erreur: {e}")
    print(f"Code de retour: {e.returncode}")
    if e.stdout:
        print(f"Sortie standard:\n{e.stdout}")
    if e.stderr:
        print(f"Erreur standard:\n{e.stderr}")

ConnectionError: Couldn't reach 'cifar10' on the Hub (LocalEntryNotFoundError)

In [ ]:
## Instructions pour utiliser votre propre dataset

### 1. Préparez votre dataset au format COCO

Organisez vos données comme suit :
```
mon_dataset/
├── images/
│   ├── train/
│   │   ├── image1.jpg
│   │   ├── image2.jpg
│   │   └── ...
│   └── val/
│       ├── image1.jpg
│       └── ...
└── annotations/
    ├── instances_train.json
    └── instances_val.json
```

### 2. Entraînez le modèle

```bash
python tools/train_custom.py \
    --dataset-path /chemin/vers/mon_dataset \
    --output-dir ./output/mon_entrainement \
    --num-classes NOMBRE_DE_CLASSES
```

### 3. Visualisez les résultats

```bash
# Visualiser les annotations ground truth
python tools/visualize_custom.py \
    --dataset-path /chemin/vers/mon_dataset \
    --output-dir ./visualizations \
    --mode gt

# Comparer avec les prédictions du modèle
python tools/visualize_custom.py \
    --dataset-path /chemin/vers/mon_dataset \
    --output-dir ./visualizations \
    --mode compare \
    --model-config ./output/mon_entrainement/config.yaml \
    --model-weights ./output/mon_entrainement/model_final.pth
```

### 4. Utilisation du script tout-en-un

```bash
python tools/run_custom_pipeline.py \
    --workspace ./mon_workspace \
    --num-train 1000 \
    --num-val 200 \
    --num-classes NOMBRE_DE_CLASSES
```

### Notes importantes :

- **Format COCO** : Vos annotations doivent être au format COCO JSON
- **Classes** : Ajustez `--num-classes` selon votre dataset
- **GPU** : L'entraînement nécessite une GPU pour des performances optimales
- **Durée** : L'entraînement complet peut prendre plusieurs heures selon la taille du dataset

### Fichiers modifiés dans le projet :

- ✅ **Support de datasets personnalisés** ajouté
- ✅ **Scripts d'entraînement** adaptés pour vos données
- ✅ **Outils de visualisation** pour analyser les résultats
- ✅ **Configuration automatique** du modèle
- ✅ **Pipeline complet** de bout en bout